# Global Fairness Verification example

Based on Calzavara et al. Explainable Global Fairness Veriﬁcation of Tree-Based Classiﬁers" (2022).

This example focuses on fairness that expresses (a lack of) causal discrimination. It is a global fairness (or verification of global robustness) task which means that is does not rely on the choice of a specific test set, but (implicitly) ranges over all possible instances.

## Preliminaries

In [1]:
%load_ext autoreload
%autoreload 2

import time
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

## German dataset

We will use the [German Credit Data](https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data).

### Load the data

In [2]:
X_df, y_df = datasets.fetch_openml(data_id=31, return_X_y=True, as_frame=True)
X, yn = datasets.fetch_openml(data_id=31, return_X_y=True, as_frame=False)

Make the target numerical

In [3]:
y = (yn=='good').astype(int)

We will focus on sex as a relevant parameter, thus will split up column 9 (index 8) 'personal_status'.

In [4]:
# weird categories, so simplify to sex
featmap = [0, 1, 0, 0, 1]
X[:, 8] = [featmap[int(val)] for val in X[:, 8]]

Introduce bias: we can optionally introduce bias to make the effect easier to see.

In [5]:
introduce_bias = 100
if introduce_bias > 0:
    X[0:introduce_bias, :] = X[0, :]
    half_idx = int(introduce_bias/2)
    X[:half_idx, 8] = 0
    y[:half_idx] = 1
    X[half_idx:introduce_bias, 8] = 1
    y[half_idx:introduce_bias] = 0

Use train and test set

In [6]:
xtrain, xtest, ytrain, ytest, ytrain_n, ytest_n = train_test_split(X, y, yn, test_size=0.2, shuffle=True)

### Train the classifier

In [7]:
import veritas
import xgboost as xgb
import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

The original paper uses 12 trees of depth 6.

In [8]:
n_estimators = 12 # 50
params = {
    "n_estimators": n_estimators,
    #"num_class": 10,
    #"objective": "multi:softmax",
    #"eval_metric": "merror",
    "eval_metric": "error",
    
    "tree_method": "hist",
    "seed": 135,
    "max_depth": 6,  # 7,
    "learning_rate": 0.2,
    "colsample_bynode": 0.75,
    "subsample": 0.5
}
model = xgb.XGBClassifier(**params)

t = time.time()
model.fit(X, y)
print(f"XGB trained in {time.time()-t:.2f} seconds")

XGB trained in 0.02 seconds


In [9]:
ytrain_pred = model.predict(xtrain)
ytest_pred = model.predict(xtest)
acc_train = accuracy_score(ytrain, ytrain_pred)
acc_test = accuracy_score(ytest, ytest_pred)

print(f"Train acc: {acc_train:.3f}, test acc: {acc_test:.3f} wrt true labels")

Train acc: 0.874, test acc: 0.850 wrt true labels


### Global fairness

The features sex is considered sensitive. Who are the subset of people whose credit risk prediction does not change by ﬂipping their sex. This maps to a query where we want to contrast two instances.

Start with loading the trained model, veritas figures out automatically it is a XGB model.

In [10]:
at = veritas.get_addtree(model)

base_score according to XGB: 0.5
base_score set to 0.0 if base_score was 0.5


For two instances X0 and X1, allowing only feature 9 (index 8) 'personal_status' to be different between the two instances, what is the maximum output difference at(X1)-at(X0)?
The values that occur are 'male single', 'female div/dep/mar', 'male div/sep' and 'male mar/wid'. Thus only the 2nd (index 1) is female.

In [11]:
nonfixed_cols = {'personal_status'}
feat_map = veritas.FeatMap(X_df.columns)
for col in X_df.columns:
    if col not in nonfixed_cols:
        feat_map.use_same_id_for(feat_map.get_index(col, 0),
                                 feat_map.get_index(col, 1))
    else:
        print(f"feat_id used for {col} for instances:",
                feat_map.get_feat_id(col, 0),
                feat_map.get_feat_id(col, 1))

at_contrast = at.concat_negated(feat_map.transform(at, 1))

feat_id used for personal_status for instances: 8 28


In [12]:
config = veritas.Config(veritas.HeuristicType.MAX_OUTPUT)
s = config.get_search(at_contrast)
duration = 10*60  # seconds
save_steps = 10
s.step_for(duration, save_steps)

<StopReason.OPTIMAL: 4>

In [26]:
print("Maximum difference between instance0 and instance1")
if s.num_solutions() > 0:
    sol = s.get_solution(0)
    print("- current best solution:", sol.output, "->",
          "optimal" if s.is_optimal() else "suboptimal", "solution")
    # print("- feature value ranges", sol.box())
    print("- feature value ranges:")
    for key, interval in sol.box().items():
        print(f"  {feat_map.get_name(key):>24}: {interval}")

Maximum difference between instance0 and instance1
- current best solution: 4.815625950000001 -> optimal solution
- feature value ranges:
           checking_status: Interval(<1)
                  duration: Interval(<10)
            credit_history: Interval(>=4)
                   purpose: Interval(>=2)
             credit_amount: Interval(866,1199)
            savings_status: Interval(>=4)
                employment: Interval(>=4)
    installment_commitment: Interval(>=4)
           personal_status: Interval(<1)
             other_parties: Interval(<2)
           residence_since: Interval(>=2)
        property_magnitude: Interval(<1)
                       age: Interval(>=67)
       other_payment_plans: Interval(>=1)
                   housing: Interval(1,2)
            num_dependents: Interval(<2)
             own_telephone: Interval(>=1)
           personal_status: Interval(>=1)


In [76]:
new_instance = np.zeros((1, 20))
for i in range(20):
    if i in sol.box():
        interval = sol.box()[i]
        value = interval.hi - 1 if np.isinf(interval.lo) else interval.lo
        new_instance[0,i] = value
    else:
        new_instance[0,i] = 0

# new_instance = np.array([[0, 5, 4, 2, 1000, 4, 4, 4, pers_status, 1, 2, 0, 70, 1, 1, 0, 0, 0, 1, 0]])
for col in nonfixed_cols:
    for nb in (0, 1):
        i = feat_map.get_feat_id(col, nb)
        if i in sol.box():
            interval = sol.box()[i]
            value = interval.hi - 1 if np.isinf(interval.lo) else interval.lo
            print(f"Non fixed column {col} gets value {value}")
            i_mapped = X_df.columns.get_loc(col)
            new_instance[0,i_mapped] = value
        print(f"Instance: {new_instance}")
        prediction = model.predict(new_instance)[0]
        print(f"==> Prediction is {prediction}")

Non fixed column personal_status gets value 0.0
Instance: [[  0.   9.   4.   2. 866.   4.   4.   4.   0.   1.   2.   0.  67.   1.
    1.   0.   0.   1.   1.   0.]]
==> Prediction is 1
Non fixed column personal_status gets value 1.0
Instance: [[  0.   9.   4.   2. 866.   4.   4.   4.   1.   1.   2.   0.  67.   1.
    1.   0.   0.   1.   1.   0.]]
==> Prediction is 0
